In [43]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import typing_extensions
import psycopg
import pandas as pd
from IPython.display import clear_output
import re

In [3]:
import sys
!{sys.executable} -m pip install ipython


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [21]:
## connection block for pulling data from our server
conn_params = {
    "dbname": "silverbugs_db",
    "user": "elias_m",
    "password": "KSefPQeAhZJZM2x7jTdDHFT8i2gwGcnC",
    "host": "dpg-d1rfk1emcj7s73e3689g-a.oregon-postgres.render.com",
    "port": 5432
}

In [101]:
with psycopg.connect(**conn_params) as conn:
    df = pd.read_sql("SELECT id, title, selftext, created_utc FROM post_data;", conn)

/var/folders/0n/7phqtgwx3t3fvjqxcbzqtft80000gn/T/ipykernel_50955/2215386082.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT id, title, selftext, created_utc FROM post_data;", conn)


In [103]:
df['silver_label'] = None
df['title+text'] = df['title'].astype('str') + df['selftext'].astype('str')
df

,id,title,selftext,created_utc,silver_label,title+text
0,1m1tfkg,[WTS] Yam’s Melt Madness (Under Melt Pre33 Gol...,**Security** Don’t share your passwords!\n\n\n...,2025-07-17 00:33:10,None,[WTS] Yam’s Melt Madness (Under Melt Pre33 Gol...
1,1m1obbm,[WTS] You Know the Deals!,**I WILL NEVER INITIATE CHAT OR MESSAGE YOU FI...,2025-07-16 20:56:02,None,[WTS] You Know the Deals!**I WILL NEVER INITIA...
2,1m1w9l7,[WTB] 1992 1/10 AGE,Looking to buy a 1/10 ounce AGE. Can pay via V...,2025-07-17 02:48:21,None,[WTB] 1992 1/10 AGELooking to buy a 1/10 ounce...
3,1m1xitf,[WTB] Series 3 Perth Lunar Gold 1oz,Looking for series 3 Perth lunar gold 1oz pref...,2025-07-17 03:52:39,None,[WTB] Series 3 Perth Lunar Gold 1ozLooking for...
4,1lxdoq8,[wts] Silver grams lot,Proof https://imgur.com/a/ONjjfGp\n\n120 Valca...,2025-07-11 17:57:21,None,[wts] Silver grams lotProof https://imgur.com/...
...,...,...,...,...,...,...
1305,1ln0ujv,"[WTS] Gold, Platinum, & Palladium.",Zelle Venmo PPFF\n\nEverything Sigma Verified\...,2025-06-28 23:49:36,None,"[WTS] Gold, Platinum, & Palladium.Zelle Venmo ..."
1306,1m0nl52,[WTS] 1/10oz Gold: Maples & Brittania + 1oz Si...,Happy Tuesday. A little fractional gold and s...,2025-07-15 17:12:09,None,[WTS] 1/10oz Gold: Maples & Brittania + 1oz Si...
1307,1lz452s,"[WTS] 40%, 90%, and ASEs","Hello and welcome! Being my first sale, I of c...",2025-07-13 21:16:26,None,"[WTS] 40%, 90%, and ASEsHello and welcome! Bei..."
1308,1lz37ed,[WTS] 40% Halves Under Spot and Germania Mint ...,Good Evening!\n\nFor sale are:\n\nqty 22 - 40%...,2025-07-13 20:37:50,None,[WTS] 40% Halves Under Spot and Germania Mint ...


### Labeling posts for bert

Bert is a powerful natural language processor. We are going to give bert some manually labeled posts to help it identify silver related posts. 

In [109]:
df.columns

Index(['id', 'title', 'selftext', 'created_utc', 'silver_label', 'title+text'], dtype='object')

In [111]:
keywords = ['silver', 'ag', '90 percent', '90%', 'junk', 'constitutional', 'silver', 'liberties', 'walkers', 'mercs', 'mercuries', 'franklins', 'walkers', 'washingtons', 'washies', 'ASE', 'ASEs', 'roosevelts']


In [113]:
pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in keywords) + r')\b', flags=re.IGNORECASE)

In [115]:
pattern = re.compile(r'(' + '|'.join(keywords) + r')', re.IGNORECASE)

In [117]:
def label_silver_posts(text):
    if pattern.search(text):
        return 1
    else:
        return 0

# Apply the function to create a new column 'silver_label'
df['silver_label'] = df['title+text'].apply(label_silver_posts)


In [119]:
df.iloc[8].selftext

'5 Gemstone multicolored bracelet, marked "Italy", "G8" and "417" (10kt), weighs 8.2g about 8" long.\nAsking $400 OBO\n\n2000 Heart necklace marked 10kt and Italy, weighs 3.8g about 16" long.\nAsking $230 OBO\n\n3 pendants;\nCentaur, weighs 1.5g unmarked $50 or free with either necklace.\nHeart weighs .6g, 14kt $35\nCross with gemstones (likely diamond) weighs 1.3g $55\n\n2 rings; \nOrnate vintage with broken stone weighs 3.4g, 10kt $140\nDented plain ring .9g 10kt $35\n\nEastern star pin, .6g 10kt $20\n\nOpen to offers and trades. \n\n\nhttps://imgur.com/gallery/kxMDt8P'

In [121]:
df['silver_label'].value_counts()

silver_label
1    1149
0     161
Name: count, dtype: int64

In [123]:
spot_pattern = re.compile(r'\b(under|below)\s+spot\b', re.IGNORECASE)

# Apply to your DataFrame
df['spot_deal'] = df.apply(
    lambda row: 1 if row['silver_label'] == 1 and bool(spot_pattern.search(row['title+text'])) else 0,
    axis=1
)

In [125]:
df

,id,title,selftext,created_utc,silver_label,title+text,spot_deal
0,1m1tfkg,[WTS] Yam’s Melt Madness (Under Melt Pre33 Gol...,**Security** Don’t share your passwords!\n\n\n...,2025-07-17 00:33:10,1,[WTS] Yam’s Melt Madness (Under Melt Pre33 Gol...,0
1,1m1obbm,[WTS] You Know the Deals!,**I WILL NEVER INITIATE CHAT OR MESSAGE YOU FI...,2025-07-16 20:56:02,1,[WTS] You Know the Deals!**I WILL NEVER INITIA...,0
2,1m1w9l7,[WTB] 1992 1/10 AGE,Looking to buy a 1/10 ounce AGE. Can pay via V...,2025-07-17 02:48:21,1,[WTB] 1992 1/10 AGELooking to buy a 1/10 ounce...,0
3,1m1xitf,[WTB] Series 3 Perth Lunar Gold 1oz,Looking for series 3 Perth lunar gold 1oz pref...,2025-07-17 03:52:39,0,[WTB] Series 3 Perth Lunar Gold 1ozLooking for...,0
4,1lxdoq8,[wts] Silver grams lot,Proof https://imgur.com/a/ONjjfGp\n\n120 Valca...,2025-07-11 17:57:21,1,[wts] Silver grams lotProof https://imgur.com/...,0
...,...,...,...,...,...,...,...
1305,1ln0ujv,"[WTS] Gold, Platinum, & Palladium.",Zelle Venmo PPFF\n\nEverything Sigma Verified\...,2025-06-28 23:49:36,1,"[WTS] Gold, Platinum, & Palladium.Zelle Venmo ...",0
1306,1m0nl52,[WTS] 1/10oz Gold: Maples & Brittania + 1oz Si...,Happy Tuesday. A little fractional gold and s...,2025-07-15 17:12:09,1,[WTS] 1/10oz Gold: Maples & Brittania + 1oz Si...,0
1307,1lz452s,"[WTS] 40%, 90%, and ASEs","Hello and welcome! Being my first sale, I of c...",2025-07-13 21:16:26,1,"[WTS] 40%, 90%, and ASEsHello and welcome! Bei...",0
1308,1lz37ed,[WTS] 40% Halves Under Spot and Germania Mint ...,Good Evening!\n\nFor sale are:\n\nqty 22 - 40%...,2025-07-13 20:37:50,1,[WTS] 40% Halves Under Spot and Germania Mint ...,1
